# Regression

In [2]:
import numpy
import urllib
import scipy.optimize
import random
import csv

In [37]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)

In [38]:
print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse255/data/beer/beer_50000.json"))
print "done"

Reading data...
done


In [40]:
print data[1]['review/taste']

3.0


### 1. Use the year ('review/timeStruct'/'year') to predict the overall rating

In [41]:
def feature(datum):
    feat = [1]
    feat.append(datum['review/timeStruct']['year'])
    return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)
print 'theta_0 = ',theta[0]
print 'theta_1 = ', theta[1]
print 'MSE = ', residuals[0]/len(y)

theta_0 =  -39.1707489355
theta_1 =  0.0214379785639
MSE =  0.490043819858


### 2. A better representation of the year variable

In [42]:
yearList = list()
for i in xrange(len(data)):
    tmpY = data[i]['review/timeStruct']['year']
    if tmpY not in yearList:
        yearList.append(tmpY)
yearList = sorted(yearList)
print yearList

def feature(datum):
    feat = [1]
    for i in range(13):
        if datum['review/timeStruct']['year'] == (yearList[i]):
            feat.append(yearList[i]-1998)
        else:
            feat.append(0)
    return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]

theta,residuals,rank,s = numpy.linalg.lstsq(X, y)
for i in range(len(theta)):
    print 'theta_',i,' = ',theta[i]
print 'MSE = ', residuals[0]/len(y)

[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
theta_ 0  =  4.05154639175
theta_ 1  =  0.0484536082475
theta_ 2  =  0.133317713214
theta_ 3  =  0.0749231325737
theta_ 4  =  -0.0731834439493
theta_ 5  =  -0.0672219161792
theta_ 6  =  -0.0485172256296
theta_ 7  =  -0.0273330482842
theta_ 8  =  -0.0305483798005
theta_ 9  =  -0.0228298815508
theta_ 10  =  -0.0160672056646
theta_ 11  =  -0.0118308912142
theta_ 12  =  -0.0128947135945
theta_ 13  =  -0.00857119748488
MSE =  0.489151895211


### 3. Train a regressor that uses the first 11 features to predict the last feature (‘quality’)

In [3]:
wine_data = []

with open('winequality-white.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    firstline = True
    for row in reader:
        if firstline:
            firstline = False
            continue
        wine_data.append([float(d) for d in row[0].split(';')])

# random.shuffle(wine_data)

In [4]:
def wine_feature(datum):
    feat = [1]
    feat += datum[:11]
    return feat

train_data = wine_data[:len(wine_data)/2]
test_data = wine_data[len(wine_data)/2:]

train_X = [wine_feature(d) for d in train_data]
train_y = [d[11] for d in train_data]

test_X = [wine_feature(d) for d in test_data]
test_y = [d[11] for d in test_data]

theta,residuals,rank,s = numpy.linalg.lstsq(train_X, train_y)
for i in range(len(theta)):
    print 'theta_'+str(i)+' = ',theta[i]
print 'Training MSE = ', residuals[0]/len(train_y)

theta_0 =  256.420279081
theta_1 =  0.135421303397
theta_2 =  -1.72994865552
theta_3 =  0.102651151592
theta_4 =  0.109038568107
theta_5 =  -0.27677514636
theta_6 =  0.00634332168497
theta_7 =  3.85023977417e-05
theta_8 =  -258.652809072
theta_9 =  1.19540565582
theta_10 =  0.83300628521
theta_11 =  0.097930435274
Training MSE =  0.602307502903


In [5]:
def compute_MSE(a,X,y):
    error = 0.0
    length = len(y)
    for i in range(length):
        error += (y[i]-a[0]-numpy.dot(a[1:],X[i][1:])) ** 2
    return error/length

In [6]:
train_MSE = compute_MSE(theta,train_X,train_y)
test_MSE = compute_MSE(theta,test_X,test_y)
print 'Training MSE = ', train_MSE
print 'Testing MSE = ', test_MSE

Training MSE =  0.602307502903
Testing MSE =  0.562457130315


### 4. An ablation experiment

In [7]:
def ablation_feature(datum,ex):
    feat = [1]
    if ex == 0:
        feat += datum[1:11]
    elif ex == 10:
        feat += datum[:10]
    else:
        feat += datum[:ex]
        feat += datum[ex+1:11]
    return feat

name = ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides",
        "free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"]

error_set = []

for ex in range(0,11):
    
    train_X = [ablation_feature(d,ex) for d in train_data]
    train_y = [d[11] for d in train_data]

    test_X = [ablation_feature(d,ex) for d in test_data]
    test_y = [d[11] for d in test_data]

    theta,residuals,rank,s = numpy.linalg.lstsq(train_X, train_y)
    
    train_MSE = compute_MSE(theta,train_X,train_y)
    test_MSE = compute_MSE(theta,test_X,test_y)
    error_set.append(abs(test_MSE-0.562457130315))
    
    print 'removing feature:', name[ex]
    print 'Testing MSE = ', test_MSE
    print
    
min_ft = numpy.argmin(error_set)
max_ft = numpy.argmax(error_set)
print 'feature with the most additional information: ', name[max_ft]
print 'feature with least additional informaiton: ', name[min_ft]

removing feature: fixed acidity
Testing MSE =  0.559113414376

removing feature: volatile acidity
Testing MSE =  0.596384850162

removing feature: citric acid
Testing MSE =  0.562221702812

removing feature: residual sugar
Testing MSE =  0.553625063967

removing feature: chlorides
Testing MSE =  0.562629266481

removing feature: free sulfur dioxide
Testing MSE =  0.55614081793

removing feature: total sulfur dioxide
Testing MSE =  0.562429005469

removing feature: density
Testing MSE =  0.544726553466

removing feature: pH
Testing MSE =  0.559566626382

removing feature: sulphates
Testing MSE =  0.557346349988

removing feature: alcohol
Testing MSE =  0.573214743558

feature with the most additional information:  volatile acidity
feature with least additional informaiton:  total sulfur dioxide


### 5. SVM classifier

In [9]:
from sklearn import svm

X_train = [wine_feature(d) for d in train_data]
y_train = [d[11]>5 for d in train_data]

X_test = [wine_feature(d) for d in test_data]
y_test = [d[11]>5 for d in test_data]

clf = svm.SVC(C=1000)
clf.fit(X_train, y_train)

train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)

In [10]:
def compute_accuracy(predictions,reality):
    length = len(reality)
    correct = 0
    for idx in range(length):
        if predictions[idx] == reality[idx]:
            correct += 1
    return (correct*1.0)/(length*1.0)

print 'Accuracy of train data = ', compute_accuracy(train_predictions,y_train)
print 'Accuracy of test data = ', compute_accuracy(test_predictions,y_test)

Accuracy of train data =  1.0
Accuracy of test data =  0.659452837893


### 6. Logistic regression

In [11]:
from math import exp
from math import log

In [39]:
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
#     print "ll =", loglikelihood
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    for i in range(len(X)):
    # Fill in code for the derivative
        logit = inner(X[i], theta)
        for k in range(len(X[i])):
            if not y[i]:
                dl[k] -= X[i][k]
            dl[k] += X[i][k]*(1.0-sigmoid(logit))
    for k in range(len(theta)):
        dl[k] -=  2*lam*theta[k]
    # Negate the return value since we're doing gradient *ascent*
    return numpy.array([-x for x in dl])

In [40]:
def wine_feature(datum):
#     feat = [1]
    feat = datum[:11]
    return feat

def compute_accuracy(test,label,theta):
    count = 0
    for i in xrange(len(test)):
        l = 0
        s = numpy.dot(test[i],theta)
        if (s > 0):
            l = 1
        if ( l == label[i]):
            count += 1
    return (count * 1.0) / (len(label) * 1.0)


n = len(wine_data)

X_train = [wine_feature(d) for d in wine_data[:n/2]]# Extract features and labels from the data
y_train = [d[11]>5 for d in wine_data[:n/2]]

X_test = [wine_feature(d) for d in wine_data[n/2:]]
y_test = [d[11]>5 for d in wine_data[n/2:]]


# X_train = X[:len(X)/2]
# X_test = X[len(X)/2:]

# If we wanted to split with a validation set:
#X_valid = X[len(X)/2:3*len(X)/4]
#X_test = X[3*len(X)/4:]

# Use a library function to run gradient descent (or you can implement yourself!)
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X_train[0]), fprime, args = (X_train, y_train, 1.0))
print "Final log likelihood =", -l
# predictions = [ for ]
print "Accuracy = ", compute_accuracy(X_test,y_test,theta)# Compute the accuracy

Final log likelihood = -1388.69601353
Accuracy =  0.76929358922


In [26]:
print theta

[  1.45226173e-04   5.22127320e-06   7.28036504e-06   1.13443530e-04
   8.94378700e-07   7.11479378e-04   2.82955820e-03   2.04293012e-05
   6.65296784e-05   1.02356116e-05   2.19267542e-04]
